In [1]:
# pip install geopy 
# pip install Nominatim

In [1]:
import pandas as pd
import time
import json
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

### Indeed Jobs

In [4]:
path_to_jobs = ('../resources/cpj_salary_location_cleaned.csv')

In [5]:
# Read raw data
jobs = pd.read_csv(path_to_jobs)
jobs.head()

,Job Title,Company Name,Location,Salary,Description,Category,Skills,Tools,Min-Salary-Range,Max-Salary-Range,Mean_Salary,City,state,zipcode
0,Senior Business Analyst - Land,Berks Homes,"Mohnton, PA 19540",NaN,Senior Business Analyst - Land Berks Homes is ...,Data Analyst,,"R, Scala",0,0,0,Mohnton,PA,19540
1,ISS-P Data Center Analyst,Consolidated Nuclear Security,"Amarillo, TX",NaN,"Location: Amarillo, TX\nJob Title: ISS-P Data ...",Data Analyst,,R,0,0,0,Amarillo,TX,NaN
2,Junior Data Analyst,Strategic Marketing Services,"Louisville, KY 40223","32,000 - 39,500",Basic Function: Develop and process targeted m...,Data Analyst,Data Analysis,"SQL, R, Excel",32000,39500,35750,Louisville,KY,40223
3,Data Analyst (Louisiana),Tetra Tech Inc.,"Laplace, LA",NaN,Tetra Tech is a leading provider of consulting...,Data Analyst,Data Analysis,"R, Excel",0,0,0,Laplace,LA,NaN
4,Marketing Data Analyst,Mercedes Scientific,"Sarasota, FL 34211","85,000",Mercedes Scientific is a national leader in th...,Data Analyst,,R,85000,0,42500,Sarasota,FL,34211


In [9]:
jobs=jobs.dropna(subset=['City','state'])
jobs

,Job Title,Company Name,Location,Salary,Description,Category,Skills,Tools,Min-Salary-Range,Max-Salary-Range,Mean_Salary,City,state,zipcode
0,Senior Business Analyst - Land,Berks Homes,"Mohnton, PA 19540",NaN,Senior Business Analyst - Land Berks Homes is ...,Data Analyst,,"R, Scala",0,0,0,Mohnton,PA,19540
1,ISS-P Data Center Analyst,Consolidated Nuclear Security,"Amarillo, TX",NaN,"Location: Amarillo, TX\nJob Title: ISS-P Data ...",Data Analyst,,R,0,0,0,Amarillo,TX,NaN
2,Junior Data Analyst,Strategic Marketing Services,"Louisville, KY 40223","32,000 - 39,500",Basic Function: Develop and process targeted m...,Data Analyst,Data Analysis,"SQL, R, Excel",32000,39500,35750,Louisville,KY,40223
3,Data Analyst (Louisiana),Tetra Tech Inc.,"Laplace, LA",NaN,Tetra Tech is a leading provider of consulting...,Data Analyst,Data Analysis,"R, Excel",0,0,0,Laplace,LA,NaN
4,Marketing Data Analyst,Mercedes Scientific,"Sarasota, FL 34211","85,000",Mercedes Scientific is a national leader in th...,Data Analyst,,R,85000,0,42500,Sarasota,FL,34211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15706,Senior Business Intelligence Analyst (Remote),Rad Power Bikes,"Seattle, WA 98107 (Ballard area)•Remote",NaN,NaN,Business Intelligence Developer,,,0,0,0,Seattle,WA,98107
15707,Sr. Business Intelligence Developer,CoreLogic,"Irving, TX 75063 (Freeport/Hackberry area)",NaN,NaN,Business Intelligence Developer,,,0,0,0,Irving,TX,75063
15709,Senior Business Intelligence Developer,BlueVine,"Redwood City, CA 94063 (Downtown area)",NaN,NaN,Business Intelligence Developer,,,0,0,0,Redwood City,CA,94063
15711,Sr. Business Intelligence Developer,Ensemble Health Partners,"Cincinnati, OH",NaN,NaN,Business Intelligence Developer,,,0,0,0,Cincinnati,OH,NaN


### Getting Latitude & Longitude

In [10]:
# Create new DF which will have lat and long
locations_df = pd.DataFrame(columns = ['location', 'city', 'state', 'latitude', 'longitude'])

coordinates = []

# Geolocator to get coordinates
geolocator = Nominatim(user_agent="JobMapping")

# Creating 'coordinates' list ['city, state']
for index in jobs.index:
    location = jobs['City'][index] + ", " + jobs['state'][index]
    coordinates.append(location)
    
coordinates_unique = list(set(coordinates))
count = 0

# Getting lat and lon for 'coordinates_unique' 
# AND store lat into 'latitude' list & lon into 'longitude' list
for loc in coordinates_unique:
        location = loc.split(', ')[0] + ", " + loc.split(', ')[1] + ", " + "US"
        print(str(count)+':\t' +location, end='')
        count += 1
        try:
            coord = geolocator.geocode(location, timeout=None)
            if(coord != None):
                locations_df.loc[len(locations_df.index)] = [loc, loc.split(', ')[0], loc.split(', ')[1],coord.latitude,coord.longitude]
                print('\t Good!')
            else:
                print('\t Empty Record!')
        except GeocoderTimedOut as e:
            print("Error: geocode failed on input %s with message %s"%(location, e.message))
            pass
        time.sleep(0.5)

0:	Breinigsville,  PA, US	 Good!
1:	Maryland Heights,  MO, US	 Good!
2:	Henderson,  NV, US	 Good!
3:	Sun Lakes,  AZ, US	 Good!
4:	Mishawaka,  IN, US	 Good!
5:	Denton,  TX, US	 Good!
6:	Clinton,  MA, US	 Good!
7:	Luke AFB,  AZ, US	 Good!
8:	West Hollywood,  CA, US	 Good!
9:	Albuquerque,  NM, US	 Good!
10:	Lees Summit,  MO, US	 Empty Record!
11:	College Park,  MD, US	 Good!
12:	Mount Carmel,  CT, US	 Good!
13:	Wright-Patterson AFB,  OH, US	 Good!
14:	Pullman,  WA, US	 Good!
15:	Dublin,  OH, US	 Good!
16:	Oakland,  CA, US	 Good!
17:	Indian Trail,  NC, US	 Good!
18:	Nutley,  NJ, US	 Good!
19:	Marysville,  OH, US	 Good!
20:	Arden Hills,  MN, US	 Good!
21:	Tarrytown,  NY, US	 Good!
22:	North Uxbridge,  MA, US	 Good!
23:	Riverwoods,  IL, US	 Good!
24:	Woodcliff Lake,  NJ, US	 Good!
25:	Denver,  CO, US	 Good!
26:	Ontario,  CA, US	 Good!
27:	Roswell,  GA, US	 Good!
28:	Chesterfield,  MO, US	 Good!
29:	Boise,  ID, US	 Good!
30:	Cape Girardeau,  MO, US	 Good!
31:	Richmond,  CA, US	 Good!
32:	Tust

264:	Temple Terrace,  FL, US	 Good!
265:	Honolulu,  HI, US	 Good!
266:	Altamahaw,  NC, US	 Good!
267:	Rancho Cucamonga,  CA, US	 Good!
268:	Bloomfield,  CT, US	 Good!
269:	Anderson,  SC, US	 Good!
270:	Nashua,  NH, US	 Good!
271:	Grand Prairie,  TX, US	 Good!
272:	Santa Barbara,  CA, US	 Good!
273:	Blair,  NE, US	 Good!
274:	Portland,  OR, US	 Good!
275:	Fate,  TX, US	 Good!
276:	Eagleville,  PA, US	 Good!
277:	Northbrook,  IL, US	 Good!
278:	Edison,  NJ, US	 Good!
279:	Arcadia,  CA, US	 Good!
280:	Elmhurst,  IL, US	 Good!
281:	Beverly Hills,  CA, US	 Good!
282:	Fremont,  CA, US	 Good!
283:	Tysons,  VA, US	 Good!
284:	Philadelphia,  PA, US	 Good!
285:	Buffalo,  NY, US	 Good!
286:	Sterling Heights,  MI, US	 Good!
287:	Glen Echo,  MD, US	 Good!
288:	Melbourne,  FL, US	 Good!
289:	Laredo,  TX, US	 Good!
290:	Stamford,  CT, US	 Good!
291:	Grapevine,  TX, US	 Good!
292:	South Jordan,  UT, US	 Good!
293:	Fulton,  MO, US	 Good!
294:	Draper,  UT, US	 Good!
295:	South Portland,  ME, US	 Good!
2

527:	Latham,  NY, US	 Good!
528:	San Bernardino,  CA, US	 Good!
529:	Burlington,  VT, US	 Good!
530:	Cedar Rapids,  IA, US	 Good!
531:	Falls Church,  VA, US	 Good!
532:	Wayne,  NJ, US	 Good!
533:	Pahrump,  NV, US	 Good!
534:	Fort Mill,  SC, US	 Good!
535:	Columbus,  OH, US	 Good!
536:	Tracys Landing,  MD, US	 Good!
537:	Westlake,  TX, US	 Good!
538:	Jackson,  TN, US	 Good!
539:	Dallas,  TX, US	 Good!
540:	Chandler,  AZ, US	 Good!
541:	Mills River,  NC, US	 Good!
542:	Bay Minette,  AL, US	 Good!
543:	New Braunfels,  TX, US	 Good!
544:	Short Hills,  NJ, US	 Good!
545:	Somerville,  MA, US	 Good!
546:	Idaho Falls,  ID, US	 Good!
547:	Van Wert,  OH, US	 Good!
548:	Concord,  MA, US	 Good!
549:	Oklahoma City,  OK, US	 Good!
550:	Western,  NE, US	 Good!
551:	McLean,  VA, US	 Good!
552:	Westminster,  CO, US	 Good!
553:	North Palm Beach,  FL, US	 Good!
554:	Plymouth Meeting,  PA, US	 Good!
555:	Susanville,  CA, US	 Good!
556:	Carson,  CA, US	 Good!
557:	Palm Bay,  FL, US	 Good!
558:	O'Fallon,  M

788:	Armonk,  NY, US	 Good!
789:	Newton,  MA, US	 Good!
790:	New Brunswick,  NJ, US	 Good!
791:	Windsor Mill,  MD, US	 Good!
792:	Louisville,  KY, US	 Good!
793:	Niles,  IL, US	 Good!
794:	Bellingham,  WA, US	 Good!
795:	Peoria,  IL, US	 Good!
796:	Newtown Square,  PA, US	 Good!
797:	West Bloomfield Township,  MI, US	 Good!
798:	Effingham,  IL, US	 Good!
799:	Atlantic Highlands,  NJ, US	 Good!
800:	Tarrant County,  TX, US	 Good!
801:	Alhambra,  CA, US	 Good!
802:	Littleton,  CO, US	 Good!
803:	West Greenwich,  RI, US	 Good!
804:	Minneapolis,  MN, US	 Good!
805:	Covington,  KY, US	 Good!
806:	Devens,  MA, US	 Good!
807:	Danvers,  MA, US	 Good!
808:	Allentown,  PA, US	 Good!
809:	Emerald Isle,  NC, US	 Good!
810:	Binghamton,  NY, US	 Good!
811:	Monmouth Junction,  NJ, US	 Good!
812:	East Rutherford,  NJ, US	 Good!
813:	Golden,  CO, US	 Good!
814:	East Hartford,  CT, US	 Good!
815:	Anchorage,  AK, US	 Good!
816:	North East,  MD, US	 Good!
817:	Notre Dame,  IN, US	 Good!
818:	Middleboro,  

1046:	Coppell,  TX, US	 Good!
1047:	Burlingame,  CA, US	 Good!
1048:	Los Gatos,  CA, US	 Good!
1049:	Westwood,  MA, US	 Good!
1050:	Sioux Falls,  SD, US	 Good!
1051:	Largo,  FL, US	 Good!
1052:	Bethlehem,  PA, US	 Good!
1053:	Md City,  MD, US	 Good!
1054:	East Hanover,  NJ, US	 Good!
1055:	Burbank,  CA, US	 Good!
1056:	Arlington County,  VA, US	 Good!
1057:	Venice,  FL, US	 Good!
1058:	Charleston,  WV, US	 Good!
1059:	Asheville,  NC, US	 Good!
1060:	Walpole,  MA, US	 Good!
1061:	Lithia Springs,  GA, US	 Good!
1062:	Largo,  MD, US	 Good!
1063:	Carlsbad,  CA, US	 Good!
1064:	Rialto,  CA, US	 Good!
1065:	Neenah,  WI, US	 Good!
1066:	Plattsmouth,  NE, US	 Good!
1067:	China,  ME, US	 Good!
1068:	Lewes,  DE, US	 Good!
1069:	Juno Beach,  FL, US	 Good!
1070:	Research Triangle Park,  NC, US	 Good!
1071:	Pierceton,  IN, US	 Good!
1072:	Camp Hill,  PA, US	 Good!
1073:	Frankenmuth,  MI, US	 Good!
1074:	Scranton,  PA, US	 Good!
1075:	Keyport,  WA, US	 Good!
1076:	Canton,  MI, US	 Good!
1077:	Big Be

In [15]:
locations_df = locations_df.rename(columns = {'city':'City'})

In [16]:
# Merging clean_df and locations_df 
merged_df = pd.merge(jobs, locations_df, on=['City', 'state'], how='left')
merged_df.head()

,Job Title,Company Name,Location,Salary,Description,Category,Skills,Tools,Min-Salary-Range,Max-Salary-Range,Mean_Salary,City,state,zipcode,location,latitude,longitude
0,Senior Business Analyst - Land,Berks Homes,"Mohnton, PA 19540",NaN,Senior Business Analyst - Land Berks Homes is ...,Data Analyst,,"R, Scala",0,0,0,Mohnton,PA,19540,"Mohnton, PA",40.285925,-75.984380
1,ISS-P Data Center Analyst,Consolidated Nuclear Security,"Amarillo, TX",NaN,"Location: Amarillo, TX\nJob Title: ISS-P Data ...",Data Analyst,,R,0,0,0,Amarillo,TX,NaN,"Amarillo, TX",35.207219,-101.833825
2,Junior Data Analyst,Strategic Marketing Services,"Louisville, KY 40223","32,000 - 39,500",Basic Function: Develop and process targeted m...,Data Analyst,Data Analysis,"SQL, R, Excel",32000,39500,35750,Louisville,KY,40223,"Louisville, KY",38.254238,-85.759407
3,Data Analyst (Louisiana),Tetra Tech Inc.,"Laplace, LA",NaN,Tetra Tech is a leading provider of consulting...,Data Analyst,Data Analysis,"R, Excel",0,0,0,Laplace,LA,NaN,"Laplace, LA",30.066591,-90.480085
4,Marketing Data Analyst,Mercedes Scientific,"Sarasota, FL 34211","85,000",Mercedes Scientific is a national leader in th...,Data Analyst,,R,85000,0,42500,Sarasota,FL,34211,"Sarasota, FL",27.336581,-82.530855


In [17]:
# Getting only the data I need for the map visual
transformed_df = merged_df[['Company Name', 'Category', 'Location', 'latitude', 'longitude']]
transformed_df = transformed_df.rename(columns = {'Company Name':'company_name','Category':'job_tile'})
transformed_df.head()


,company_name,job_tile,Location,latitude,longitude
0,Berks Homes,Data Analyst,"Mohnton, PA 19540",40.285925,-75.984380
1,Consolidated Nuclear Security,Data Analyst,"Amarillo, TX",35.207219,-101.833825
2,Strategic Marketing Services,Data Analyst,"Louisville, KY 40223",38.254238,-85.759407
3,Tetra Tech Inc.,Data Analyst,"Laplace, LA",30.066591,-90.480085
4,Mercedes Scientific,Data Analyst,"Sarasota, FL 34211",27.336581,-82.530855


In [18]:
indeed_jobs_jsonfile = transformed_df.to_json(orient="index")

In [19]:
# Writing the output to json file 
with open('../resources/mw_indeed_jobs_geo.json', 'w') as file:
    file.write(indeed_jobs_jsonfile)